In [1]:
# William Daniels
# A* Vacuum Search

In [ ]:
# With heuristic h1 (Manhattan Distance to nearest dirty square):

In [36]:
from search import *

class Vacuum(Problem):
    """ The problem of moving the Vacuum agent from one place to other """

    def __init__(self, initial, goal, allowed=None, dimrow=None):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)
        # self.dimrow = dimrow
        self.goal = goal
        # self.allowed = allowed

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only five possible actions
        in any given state of the environment """
        
        # There are 4 possible move actions and 1 other possible action ('Suck')
        possible_actions = ['Up', 'Down', 'Right', 'Left', 'Suck']
        
        # Get the x and y coordinates of the current node
        x, y = getLocation(state)
   
        # Assign the truth value of whether the current node is dirty or not to 'dirty'
        dirty = dirtyCheck(state, x, y)
        
        # If the current node is at the top of the grid, it is impossible to move up
        if y == 5:
            possible_actions.remove("Up")
            
        # If the current node is at the bottom of the grid, it is impossible to move down
        if y == 1:
            possible_actions.remove("Down")
            
        # If the current node is on the right edge of the grid, it is impossible to move right
        if x == 5:
            possible_actions.remove("Right")
            
        # If the current node is on the left edge of the grid, it is impossible to move left
        if x == 1:
            possible_actions.remove("Left")
            
        # If the current node is not dirty, it should not 'Suck'
        if not dirty:
            possible_actions.remove("Suck")
            
        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """
        
        # Get the x and y coordinates of the current node
        x, y = getLocation(state)
        
        # Assign a list of dirty square coordinates to 'dirtyList'
        dirtyList = getDirtyList(state)

        # Move Up
        if action == 'Up':
            # New state after moving up
            state = makeState((x,y+1), dirtyList)
            
        # Move Down
        elif action == 'Down':
            # New state after moving down
            state = makeState((x,y-1), dirtyList)
                        
        # Move Right
        elif action == 'Right':
            # New state after moving right
            state = makeState((x+1,y), dirtyList)
            
        # Move Left
        elif action == 'Left':
            # New state after moving left
            state = makeState((x-1,y), dirtyList)
            
        # Suck
        elif action == 'Suck':
            # Remove the node about to be sucked from the dirty list before making the new state
            dirtyList.remove((x,y))
            
            # New state after sucking
            state = makeState((x,y), dirtyList)
            
        return tuple(state)
    
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        
        # Get the amount of dirty squares left after the action has been taken
        amountDirty = len(getDirtyList(state2))
        
        # return the specified pathCost (number of dirty squares * 2 + 1)
        return c + amountDirty * 2 + 1
    
    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        
        # Format the state representation from a list containing (x, y, dirty, current) 
        # to a list of only (dirty), i.e. a list of every dirty value in the state
        dirtyValues = []
        for tup in state:
            dirtyValues.append(tup[2])
            
        # The goalTest is the comparison of the current total dirty squares to the goal
        return dirtyValues == self.goal

    def h(self, node):
        """ Return the heuristic value for a given state."""
        
        # Get the x and y coordinates of the current node
        x, y = getLocation(node.state)
            
        numx, numy = 0, 0
        
        # get list of dirty squares
        dirtyList = getDirtyList(node.state)
        distanceList = []
        
        # get a list of the manhattan distances of the current square to every dirty square
        for square in dirtyList:
            numx, numy = square[0], square[1]
            distanceList.append(abs(numx - x) + abs(numy - y))
        
        minMD = 0
        # get the least of the distances in the list
        if distanceList:
            minMD = min(distanceList)
        
        # The heuristic is the Manhattan distance of the current square to the closest dirty square
        return minMD
# ______________________________________________________________________________


def makeState(current, dirtyList):
    """ Makes the state based on the current location of the agent
    and the location of dirty squares. Each square is represented by 
    a tuple in the format of: 
    (square's x coord, square's y coord, squareDirty?, isAgentHere?).
    There are 25 of these smaller tuples which are packaged into one
    bigger tuple that is called the state """
    
    state = []
    # 25 squares
    for i in range(1,6):
        for j in range(1,6):
                # if the square being made should be dirty
                if (i,j) in dirtyList:
                    # if the square being made is the agent current location
                    if current == (i,j):
                        state.append((i,j,1,1))
                    else:
                        state.append((i,j,1,0))
                else:
                    # else if the square being made is the agent current location
                    if current == (i,j):
                        state.append((i,j,0,1))
                    # else (a clean, non occupied square)
                    else:
                        state.append((i,j,0,0))
    return tuple(state)

def getLocation(state):
    """ Accesses the x and y coordinates of the square that is 
    the current location of the agent """
    
    # for every square in the state
    for tup in state:
        # if the agent is in the square
        if tup[3]:
            # return x, y
            return tup[0], tup[1]

def dirtyCheck(state, x, y):
    """ Checks to see if a square is dirty by getting its dirty value """
    # for every square
    for tup in state:
        # if the coordinates match
        if x == tup[0] and y == tup[1]:
            # return the dirty value
            return tup[2]
    
def getDirtyList(state):
    """ Gets a list of every remaining dirty square """
    dirtyList = []
    
    # for every square
    for tup in state:
        # if the square is dirty
        if tup[2]:
            # add it to a list
            dirtyList.append((tup[0], tup[1]))
            
    return dirtyList

def formatPath(path):
    """ Formats the path that the agent takes. Needed because the path
    is not readable as it contains the information of every square at every
    movement. """
    
    # for every node (state of 25 squares) in the path
    for node in path:
        # some formatting
        node = str(node)[6:-1]
        
    # change every node from string to a tuple
    tupList = [eval(str(node)[6:-1]) for node in path]
    
    path = []
    # for every tuple in the list of tuples
    for tup in tupList:
        # for every smaller tuple inside of that
        for innerTup in tup:
            # if the square had an agent in it at some point and if the square isn't dirty
            # (this eliminates duplicate entries due to staying in the same place to suck)
            if innerTup[3] and not innerTup[2]:
                # add it to the path
                path.append((innerTup[0], innerTup[1]))

    # return the formatted final agent path
    return path


# --------------------------------------------------------------------------------------------------------------------
# Main
# --------------------------------------------------------------------------------------------------------------------

# the x,y coordinates 5 dirty squares as specified in the project inscructions
dirtySquares = [(1,5),(2,5),(3,5),(4,5),(5,5)]

# initialization of initial state
initial = makeState((1,1), dirtySquares)

# initialization of goal state
goal = [0] * 25

# call astar search on the Vacuum(Problem) and get the path, solution, and path cost
path = formatPath(astar_search(Vacuum(initial, goal)).path())
solution = astar_search(Vacuum(initial, goal)).solution()
pathCost = astar_search(Vacuum(initial, goal)).path_cost

# number of expanded nodes, gotten by getting length of path minus its duplicate entries
numExpanded = len(set(path))

print("Agent Path:", path)
print("Agent actions:", solution)
print("Path cost:", pathCost)
print("Number of Expanded Nodes:", numExpanded)

Agent Path: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
Agent actions: ['Up', 'Up', 'Up', 'Up', 'Suck', 'Right', 'Suck', 'Right', 'Suck', 'Right', 'Suck', 'Right', 'Suck']
Path cost: 93
Number of Expanded Nodes: 9


In [ ]:
# With heuristic h2 (Manhattan Distance to nearest dirty square + number of dirty squares):

In [37]:
from search import *


class Vacuum(Problem):
    """ The problem of moving the Vacuum agent from one place to other """

    def __init__(self, initial, goal, allowed=None, dimrow=None):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)
        # self.dimrow = dimrow
        self.goal = goal
        # self.allowed = allowed

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only five possible actions
        in any given state of the environment """
        
        # There are 4 possible move actions and 1 other possible action ('Suck')
        possible_actions = ['Up', 'Down', 'Right', 'Left', 'Suck']
        
        # Get the x and y coordinates of the current node
        x, y = getLocation(state)
   
        # Assign the truth value of whether the current node is dirty or not to 'dirty'
        dirty = dirtyCheck(state, x, y)
        
        # If the current node is at the top of the grid, it is impossible to move up
        if y == 5:
            possible_actions.remove("Up")
            
        # If the current node is at the bottom of the grid, it is impossible to move down
        if y == 1:
            possible_actions.remove("Down")
            
        # If the current node is on the right edge of the grid, it is impossible to move right
        if x == 5:
            possible_actions.remove("Right")
            
        # If the current node is on the left edge of the grid, it is impossible to move left
        if x == 1:
            possible_actions.remove("Left")
            
        # If the current node is not dirty, it should not 'Suck'
        if not dirty:
            possible_actions.remove("Suck")
            
        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """
        
        # Get the x and y coordinates of the current node
        x, y = getLocation(state)
        
        # Assign a list of dirty square coordinates to 'dirtyList'
        dirtyList = getDirtyList(state)

        # Move Up
        if action == 'Up':
            # New state after moving up
            state = makeState((x,y+1), dirtyList)
            
        # Move Down
        elif action == 'Down':
            # New state after moving down
            state = makeState((x,y-1), dirtyList)
                        
        # Move Right
        elif action == 'Right':
            # New state after moving right
            state = makeState((x+1,y), dirtyList)
            
        # Move Left
        elif action == 'Left':
            # New state after moving left
            state = makeState((x-1,y), dirtyList)
            
        # Suck
        elif action == 'Suck':
            # Remove the node about to be sucked from the dirty list before making the new state
            dirtyList.remove((x,y))
            
            # New state after sucking
            state = makeState((x,y), dirtyList)
            
        return tuple(state)
    
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        
        # Get the amount of dirty squares left after the action has been taken
        amountDirty = len(getDirtyList(state2))
        
        # return the specified pathCost (number of dirty squares * 2 + 1)
        return c + amountDirty * 2 + 1
    
    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        
        # Format the state representation from a list containing (x, y, dirty, current) 
        # to a list of only (dirty), i.e. a list of every dirty value in the state
        dirtyValues = []
        for tup in state:
            dirtyValues.append(tup[2])
            
        # The goalTest is the comparison of the current total dirty squares to the goal
        return dirtyValues == self.goal

    def h(self, node):
        """ Return the heuristic value for a given state. """
        
         # Get the x and y coordinates of the current node
        x, y = getLocation(node.state)
            
        numx, numy = 0, 0
        
        # get list of dirty squares
        dirtyList = getDirtyList(node.state)
        distanceList = []
        
        # get a list of the manhattan distances of the current square to every dirty square
        for square in dirtyList:
            numx, numy = square[0], square[1]
            distanceList.append(abs(numx - x) + abs(numy - y))
        
        minMD = 0
        # get the least of the distances in the list
        if distanceList:
            minMD = min(distanceList)
        
        # The heuristic is the Manhattan distance of the current square to the closest dirty square
        # PLUS the amount of dirty squares in the state
        return minMD + len(getDirtyList(node.state))
# ______________________________________________________________________________


def makeState(current, dirtyList):
    """ Makes the state based on the current location of the agent
    and the location of dirty squares. Each square is represented by 
    a tuple in the format of: 
    (square's x coord, square's y coord, squareDirty?, isAgentHere?).
    There are 25 of these smaller tuples which are packaged into one
    bigger tuple that is called the state """
    
    state = []
    # 25 squares
    for i in range(1,6):
        for j in range(1,6):
                # if the square being made should be dirty
                if (i,j) in dirtyList:
                    # if the square being made is the agent current location
                    if current == (i,j):
                        state.append((i,j,1,1))
                    else:
                        state.append((i,j,1,0))
                else:
                    # else if the square being made is the agent current location
                    if current == (i,j):
                        state.append((i,j,0,1))
                    # else (a clean, non occupied square)
                    else:
                        state.append((i,j,0,0))
    return tuple(state)

def getLocation(state):
    """ Accesses the x and y coordinates of the square that is 
    the current location of the agent """
    
    # for every square in the state
    for tup in state:
        # if the agent is in the square
        if tup[3]:
            # return x, y
            return tup[0], tup[1]

def dirtyCheck(state, x, y):
    """ Checks to see if a square is dirty by getting its dirty value """
    
    # for every square
    for tup in state:
        # if the coordinates match
        if x == tup[0] and y == tup[1]:
            # return the dirty value
            return tup[2]
    
def getDirtyList(state):
    """ Gets a list of every remaining dirty square """
    
    dirtyList = []
    
    # for every square
    for tup in state:
        # if the square is dirty
        if tup[2]:
            # add it to a list
            dirtyList.append((tup[0], tup[1]))
            
    return dirtyList
# ______________________________________________________________________________

def formatPath(path):
    """ Formats the path that the agent takes. Needed because the path
    is not readable as it contains the information of every square at every
    movement. """
    
    # for every node (state of 25 squares) in the path
    for node in path:
        # some formatting
        node = str(node)[6:-1]
    # change every node from string to a tuple
    tupList = [eval(str(node)[6:-1]) for node in path]
    
    path = []
    # for every tuple in the list of tuples
    for tup in tupList:
        # for every smaller tuple inside of that
        for innerTup in tup:
            # if the square had an agent in it at some point and if the square isn't dirty
            # (this eliminates duplicate entries due to staying in the same place to suck)
            if innerTup[3] and not innerTup[2]:
                # add it to the path
                path.append((innerTup[0], innerTup[1]))
    
    # return the formatted final agent path
    return path


# --------------------------------------------------------------------------------------------------------------------
# Main
# --------------------------------------------------------------------------------------------------------------------

# the x,y coordinates 5 dirty squares as specified in the project instructions
dirtySquares = [(1,5),(2,5),(3,5),(4,5),(5,5)]

# initialization of initial state
initial = makeState((1,1), dirtySquares)

# initialization of goal state
goal = [0] * 25

# call astar search on the Vacuum(Problem) and get the path, solution, and path cost
path = formatPath(astar_search(Vacuum(initial, goal)).path())
solution = astar_search(Vacuum(initial, goal)).solution()
pathCost = astar_search(Vacuum(initial, goal)).path_cost

# number of expanded nodes, gotten by getting length of path minus its duplicate entries
numExpanded = len(set(path))

print("Agent Path:", path)
print("Agent actions:", solution)
print("Path cost:", pathCost)
print("Number of Expanded Nodes:", numExpanded)

Agent Path: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
Agent actions: ['Up', 'Up', 'Up', 'Up', 'Suck', 'Right', 'Suck', 'Right', 'Suck', 'Right', 'Suck', 'Right', 'Suck']
Path cost: 93
Number of Expanded Nodes: 9
